In [1]:
import pandas as pd
import numpy as np
from datetime import date


import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go


import dash 
from dash import html, dcc
from dash.html.H1 import H1
# import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

In [2]:
confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
deaths_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
print(confirmed_df.shape, deaths_df.shape, recovered_df.shape)

(284, 825) (284, 825) (269, 825)


In [3]:
recovered_df.isna().sum()

Province/State    196
Country/Region      0
Lat                 1
Long                1
1/22/20             0
                 ... 
4/17/22             0
4/18/22             0
4/19/22             0
4/20/22             0
4/21/22             0
Length: 825, dtype: int64

In [3]:
# Drop rows that have nulls in 'Lat' and 'Long' cols
confirmed_df.dropna(subset=['Lat','Long'], inplace=True)
deaths_df.dropna(subset=['Lat','Long'], inplace=True)
recovered_df.dropna(subset=['Lat','Long'], inplace=True)

print(confirmed_df.shape, deaths_df.shape, recovered_df.shape)

(282, 825) (282, 825) (268, 825)


In [4]:
dates = confirmed_df.columns[4:]

# unpivot dataframe
# make the date cols as rows

cases = ['Confirmed', 'Deaths', 'Recovered']
dfs = []


df_names = ['confirmed_df', 'deaths_df', 'recovered_df']
for i, case in enumerate(cases):
    df_long = eval(f'{df_names[i]}').melt(
        id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
        value_vars=dates, 
        var_name='Date', 
        value_name=case
    )
    dfs.append(df_long)

# confirmed_df_long, deaths_df_long, recovered_df_long = dfs

    Province/State        Country/Region        Lat        Long  1/22/20  \
0              NaN           Afghanistan  33.939110   67.709953        0   
1              NaN               Albania  41.153300   20.168300        0   
2              NaN               Algeria  28.033900    1.659600        0   
3              NaN               Andorra  42.506300    1.521800        0   
4              NaN                Angola -11.202700   17.873900        0   
..             ...                   ...        ...         ...      ...   
279            NaN    West Bank and Gaza  31.952200   35.233200        0   
280            NaN  Winter Olympics 2022  39.904200  116.407400        0   
281            NaN                 Yemen  15.552727   48.516388        0   
282            NaN                Zambia -13.133897   27.849332        0   
283            NaN              Zimbabwe -19.015438   29.154857        0   

     1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  ...  4/12/22  4/13/22  \
0          0

In [5]:
# remove recovered data for Canada due to mismatch issue 
# Canada recovered data is counted by Country-wise rather than Province/State-wise
recovered_df_long = dfs[2]
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']
dfs[2] = recovered_df_long


In [6]:
from functools import reduce

# Merging all dataframes in dfs
full_table = reduce(lambda left,right: pd.merge(left, right, how='left', on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']), dfs)

In [11]:
full_table[full_table['Recovered']!=0].isnull().sum()

Province/State    91894
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered         16420
dtype: int64

In [31]:
# convert 'Date' values from string to datetime
full_table['Date'] = pd.to_datetime(full_table['Date'])
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0
...,...,...,...,...,...,...,...,...
231517,NaN,West Bank and Gaza,31.952200,35.233200,2022-04-21,656876,5657,0.0
231518,NaN,Winter Olympics 2022,39.904200,116.407400,2022-04-21,535,0,0.0
231519,NaN,Yemen,15.552727,48.516388,2022-04-21,11818,2148,0.0
231520,NaN,Zambia,-13.133897,27.849332,2022-04-21,318984,3974,0.0


In [10]:
full_table.isnull().sum()

Province/State    160095
Country/Region         0
Lat                    0
Long                   0
Date                   0
Confirmed              0
Deaths                 0
Recovered          16420
dtype: int64

**We found a lot NaN in Province/State, and that makes sense as many countries only report the Country-wise data. However, there are 12,315 NaNs in Recovered and let’s replace them with 0**

In [32]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)

**there are coronavirus cases reported from 3 cruise ships: Grand Princess, Diamond Princess and MS Zaandam.
These data need to be extracted and treated differently due to Province/State and Country/Region mismatch over time**

In [33]:
ship_rows = full_table['Province/State'].str.contains('Grand Princess') \
            | full_table['Province/State'].str.contains('Diamond Princess') \
            | full_table['Country/Region'].str.contains('Diamond Princess') \
            | full_table['Country/Region'].str.contains('MS Zaandam')

full_ship = full_table[ship_rows]

# full table without full_ship
full_table = full_table[~(ship_rows)]

In [34]:
# Active Case = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

C:\Users\ItShop\AppData\Local\Temp/ipykernel_14736/2634332054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']


In [35]:
full_table.describe()

,Lat,Long,Confirmed,Deaths,Recovered,Active
count,228238.000000,228238.000000,2.282380e+05,228238.000000,2.282380e+05,2.282380e+05
mean,20.395669,22.274671,5.321524e+05,10045.989217,1.016854e+05,4.204211e+05
std,25.866762,76.255799,3.069714e+06,50577.075104,8.222682e+05,2.903788e+06
min,-71.949900,-178.116500,0.000000e+00,0.000000,0.000000e+00,-1.638280e+05
25%,6.423800,-42.604300,3.080000e+02,2.000000,0.000000e+00,2.400000e+01
50%,22.041450,21.375600,6.357000e+03,86.000000,1.500000e+01,1.629000e+03
75%,41.153300,88.092400,1.205245e+05,1909.000000,3.956000e+03,3.588900e+04
max,71.706900,178.065000,8.085091e+07,990679.000000,3.097475e+07,7.986023e+07


In [14]:
# full_table[full_table['Active']<0]
# full_table.iloc[23495,:]
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
231517,NaN,West Bank and Gaza,31.952200,35.233200,2022-04-21,656876,5657,0.0,651219.0
231518,NaN,Winter Olympics 2022,39.904200,116.407400,2022-04-21,535,0,0.0,535.0
231519,NaN,Yemen,15.552727,48.516388,2022-04-21,11818,2148,0.0,9670.0
231520,NaN,Zambia,-13.133897,27.849332,2022-04-21,318984,3974,0.0,315010.0


In [36]:
# aggregate data into Country/Region wise and group them by Date and Country/Region.
full_grouped = full_table.groupby(['Date', 'Country/Region', 'Lat', 'Long'])['Confirmed', 'Deaths', 'Recovered', 'Active'] \
                .sum().reset_index()

C:\Users\ItShop\AppData\Local\Temp/ipykernel_14736/241417520.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full_grouped = full_table.groupby(['Date', 'Country/Region', 'Lat', 'Long'])['Confirmed', 'Deaths', 'Recovered', 'Active'] \


In [17]:
# add day wise New cases, New deaths and New recovered
# by deducting the corresponding accumulative data on the previous day.

# new cases 
temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan

# renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']

# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])

# filling na with 0
full_grouped = full_grouped.fillna(0)

# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')

# 
full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)


C:\Users\ItShop\AppData\Local\Temp/ipykernel_14736/1978114022.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']


In [18]:
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,2020-01-22,Afghanistan,0,0,0.0,0.0,0,0,0
1,2020-01-22,Albania,0,0,0.0,0.0,0,0,0
2,2020-01-22,Algeria,0,0,0.0,0.0,0,0,0
3,2020-01-22,Andorra,0,0,0.0,0.0,0,0,0
4,2020-01-22,Angola,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...
160911,2022-04-21,West Bank and Gaza,656876,5657,0.0,651219.0,0,0,0
160912,2022-04-21,Winter Olympics 2022,535,0,0.0,535.0,0,0,0
160913,2022-04-21,Yemen,11818,2148,0.0,9670.0,0,0,0
160914,2022-04-21,Zambia,318984,3974,0.0,315010.0,344,1,0


In [19]:
full_grouped.to_csv('COVID-19-time-series-clean-complete.csv')

In [20]:
full_grouped.describe()

,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
count,1.609160e+05,160916.000000,1.609160e+05,1.609160e+05,1.609160e+05,160916.000000,1.609160e+05
mean,7.547877e+05,14248.903074,1.442272e+05,5.963116e+05,3.157410e+03,38.603669,-1.864327e-04
std,3.639311e+06,59767.402372,9.761743e+05,3.449664e+06,1.921510e+04,185.456585,9.751155e+04
min,0.000000e+00,0.000000,0.000000e+00,-1.638280e+05,0.000000e+00,-1918.000000,-3.097475e+07
25%,1.479000e+03,21.000000,0.000000e+00,1.970000e+02,0.000000e+00,0.000000,0.000000e+00
50%,2.552200e+04,393.000000,8.000000e+01,6.619000e+03,5.800000e+01,0.000000,0.000000e+00
75%,2.646698e+05,4683.000000,1.695325e+04,8.763550e+04,7.800000e+02,10.000000,5.500000e+01
max,8.085091e+07,990679.000000,3.097475e+07,7.986023e+07,1.383823e+06,11447.000000,1.123456e+06


In [21]:
full_grouped[full_grouped['Active'] < 0]

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
10161,2020-03-13,Sudan,0,1,0.0,-1.0,0,1,0
30090,2020-06-23,Liechtenstein,79,1,81.0,-3.0,0,0,0
35462,2020-07-20,Uganda,1069,0,1071.0,-2.0,4,0,48
38570,2020-08-05,Seychelles,114,0,124.0,-10.0,0,0,11
38766,2020-08-06,Seychelles,114,0,124.0,-10.0,0,0,0
...,...,...,...,...,...,...,...,...,...
109505,2021-08-02,Peru,2114445,196518,2081755.0,-163828.0,1244,80,1331
109637,2021-08-03,Guyana,22643,549,22327.0,-233.0,120,8,1144
109701,2021-08-03,Peru,2116652,196598,2083747.0,-163693.0,2207,80,1992
109833,2021-08-04,Guyana,22643,549,22327.0,-233.0,0,0,0
